This python code has been developped by [Taïs Grippa](https://github.com/tgrippa) (Université Libre de Bruxelles). 

Code developped on Ubuntu 22.04 (Ubuntu Jammy) and GRASS GIS 8.0.2 using the Docker environment [available here](https://github.com/tgrippa/Weaksupervision_Vaihingen).

# Define working environment

**Import libraries**

In [64]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import time 
print(sys.version)

3.10.4 (main, Jun 29 2022, 12:14:53) [GCC 11.2.0]


In [ ]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
import functools
from functools import partial

**Add folder with SCR provided belong to this notebook**

In [66]:
# Add local module to the path
src = os.path.abspath('/home/tais/github/SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev)**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [67]:
run /home/tais/github/SRC/config.py

In [68]:
print(config_parameters)

{'GISBASE': '/usr/lib/grass78', 'PYTHONLIB': '/usr/bin/python3', 'locationepsg': '4326', 'gisdb': '/home/tais/GRASSDATA', 'location': 'vaihingen', 'permanent_mapset': 'PERMANENT', 'outputfolder': '/home/tais/result', 'inputdir': '/home/tais/data', 'images_val': ['3', '13', '17', '26', '32', '37'], 'images_classical_approach': ['1', '5', '7', '11', '15', '21', '23', '28', '30', '34'], 'images_snorkel_approach': ['1', '2', '4', '5', '6', '7', '8', '10', '11', '12', '14', '15', '16', '20', '21', '22', '23', '24', '27', '28', '29', '30', '31', '33', '34', '35', '38'], 'images_snorkel_approach_unlabel': ['2', '4', '6', '8', '10', '12', '14', '16', '20', '22', '24', '27', '29', '31', '33', '35', '38'], 'images_snorkel_approach_gts': ['1', '5', '7', '11', '15', '21', '23', '28', '30', '34']}


In [69]:
print(data)

{'dsm_folder': '/home/tais/data/ISPRS_semantic_labeling_Vaihingen/dsm', 'top_folder': '/home/tais/data/ISPRS_semantic_labeling_Vaihingen/top', 'gts_folder': '/home/tais/data/ISPRS_semantic_labeling_Vaihingen/gts_for_participants', 'legend': '/home/tais/github/Legend.txt'}


In [70]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [71]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

PATH	= /.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/grass78/bin:/usr/lib/grass78/script:/usr/lib/grass78/lib:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass78/bin:/usr/lib/grass78/scripts:/home/tais/.grass7/addons/scripts:/hom

**GRASS GIS Python libraries**

In [72]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Other functions**

In [88]:
from grass.script import vector
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for sorting string number naturally
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [89]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Init
    gsetup.init(config_parameters['GISBASE'], config_parameters['gisdb'], config_parameters['location'], mapset)
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [90]:
def reclass_label(image_id, folder):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='segmentation')
    # Create probability raster using r.reclass
    gscript.run_command('r.reclass', overwrite=True, input='segmentation', output='snorkel_label', rules=os.path.join(folder,'label_image_%s.txt'%image_id))
    # Change colors 
    with open(data["legend"]) as csvfile:   # Read the legend to get colors and label
        reader = csv.reader(csvfile, delimiter=';')  
        content = [['-1','0:0:0']]
        for r in reader: 
            content.append([r[0],r[1]])    
    with tempfile.NamedTemporaryFile(mode='w',prefix='rcolors',suffix='.txt',delete=False) as fp:   # Write txt rule file for r.colors
        writer = csv.writer(fp, delimiter=' ')
        writer.writerows(content)
        rule_file = fp.name
    gscript.run_command('r.colors', quiet=True, map='snorkel_label', rules=rule_file)  # Apply the color rules to the raster file

In [100]:
def reclass_proba(image_id, folder):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='segmentation')
    # Create label raster using r.reclass and r.mapcalc   
    gscript.run_command('r.reclass', overwrite=True, input='segmentation', output='tmp', rules=os.path.join(folder,'prob_image_%s.txt'%image_id))
    formula = "snorkel_proba = float(tmp/10000.0)"
    gscript.mapcalc(formula, overwrite=True)
    #gscript.run_command('g.remove', flags='f', type='raster', name='tmp')
    gscript.run_command('r.colors', quiet=True, map='snorkel_proba', color='grey')  # Apply the color rules to the raster file

In [112]:
def export(image_id, folder):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='segmentation')
    # Export labels  
    gscript.run_command('r.out.gdal', overwrite=True, input='snorkel_label', output=os.path.join(folder,'Label_image%s.tif'%image_id), format='GTiff')
    # Export probabilities  
    gscript.run_command('r.out.gdal', overwrite=True, input='snorkel_proba', output=os.path.join(folder,'Proba_image%s.tif'%image_id), format='GTiff')

In [132]:
def worker(image_id, folder_reclass='/home/tais/result/snorkel_reclass_rules/fixed_threshold', folder_export='/home/tais/result/snorkel_predicts/fixed_threshold'):
    # Launch a new mapset for this image ID
    launch_mapset("image_%s"%image_id)
    # Reclass label
    reclass_label(image_id, folder_reclass)
    # Reclass probability
    reclass_proba(image_id, folder_reclass)
    # Export 
    export(image_id, folder_export)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [133]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(os.path.join(config_parameters['outputfolder'],'raster_label'))
check_create_dir(os.path.join(config_parameters['outputfolder'],'raster_proba'))

The folder '/home/tais/result' already exists
The folder '/home/tais/result/raster_label' already exists
The folder '/home/tais/result/raster_proba' already exists


# Recode segmentation raster and export Snorkel labels and probabilities

In [140]:
# Launch processes in parallel
start_parallel = start_processing()
for x in config_parameters["images_snorkel_approach_unlabel"]:
    worker(x, '/home/tais/result/snorkel_reclass_rules/varying_threshold', '/home/tais/result/snorkel_predicts/varying_threshold')  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
    worker(x, '/home/tais/result/snorkel_reclass_rules/fixed_threshold', '/home/tais/result/snorkel_predicts/fixed_threshold')  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
# Print
print_processing_time(start_parallel, "Computation achieved in ")

Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Int16>
Exporting raster data to GTiff format...
ERROR 6: /home/tais/result/snorkel_predicts/varying_threshold/Label_image2.tif, band 1: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File
</home/tais/result/snorkel_predicts/varying_threshold/Label_image2.tif>
created.
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Float32>
Exporting raster data to GTiff format...
ERROR 6: /home/tais/result/snorkel_predicts/varying_threshold/Proba_image2.tif, band 1: SetC

'Computation achieved in 1 minutes and 19.0 seconds'